In [3]:
pip install pandas

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 60.1 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 KB 60.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 69.7 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
input_csv = "/home/shana/nelli-genomes-db/data/gvclass_genus_1_2025-08-25/genome_download_list.csv"

In [2]:
df = pd.read_csv(input_csv)

df['genome_size'] = pd.to_numeric(df['genome_size'], errors='coerce')
df['taxonomy'] = df['taxonomy'].str.lower()

Getting smallest datasets for easy testing (3 baccteria, 3 eukaryota, 3 archaea)

In [3]:
def get_smallest(group_name, n=3):
    """Return the n smallest genomes from a given taxonomic group"""
    return df[df['taxonomy'].str.contains(group_name)].nsmallest(n, 'genome_size')

bacteria = get_smallest('bacteria', 3)
archaea = get_smallest('archaea', 3)
eukarya = get_smallest('eukaryota', 3)

filtered_df = pd.concat([bacteria, archaea, eukarya])
filtered_df[['accession', 'organism', 'taxonomy', 'genome_size', 'prefix']]

,accession,organism,taxonomy,genome_size,prefix
2107,GCA_003544595.1,Bacteriophage sp.,viruses;unclassified bacterial viruses;bacteri...,3161,PHAGE__
9144,GCA_000902155.1,Enterobacteria phage M,viruses;riboviria;orthornavirae;lenarviricota;...,3405,PHAGE__
27555,GCA_031321885.1,Pinkberry virus LS07-2018-MD00,viruses;unclassified bacterial viruses;pinkber...,19623,PHAGE__
1393,GCA_027582465.1,Archaeal virus sp.,viruses;unclassified viruses;unclassified arch...,6054,PHAGE__
11604,GCA_027582475.1,Huginn virus,viruses;unclassified viruses;unclassified arch...,7373,PHAGE__
1507,GCA_029870215.1,Asgard archaea virus SkuldV2,viruses;unclassified viruses;unclassified dna ...,11871,PHAGE__
24236,GCA_902498725.1,Leishmania major strain Friedlin,cellular organisms;eukaryota;discoba;euglenozo...,2736,EUK__
9765,GCA_902825245.1,Fragilariopsis kerguelensis,cellular organisms;eukaryota;sar;stramenopiles...,10223,EUK__
5154,GCA_046252445.1,Candidozyma auris,cellular organisms;eukaryota;opisthokonta;fung...,10819,EUK__


Filtering MAGs

In [ ]:
import glob
from pathlib import Path
import shutil

fasta_dir = Path("/home/shana/nelli-genomes-db/data/gvclass_genus_1_2025-08-25/fna/")
output_dir = Path("/home/shana/nelli-genomes-db/data/gvclass_genus_1_2025-08-25/filtered-mags/")

for _, row in filtered_df.iterrows():
    accession_mod = row['accession_mod'] #extracting accession num for matching
    prefix = row['prefix']
    fasta_pattern = f"{prefix}{accession_mod}.fna" #extracting GCA-000XXXXXX format    

    fasta_files = list(fasta_dir.glob(fasta_pattern)) #look for matching fna file

    if fasta_files:
        for fasta_file in fasta_files:
            dest_file = output_dir / fasta_file.name
            
            if not dest_file.exists():
                    shutil.copy(fasta_files[0], output_dir / fasta_files[0].name)
                    print(f"Copied: {fasta_files[0].name}")
                
            else:
                print("File already exists")
    else:
        print(f"No fasta found for prefix: {prefix}{accession_mod}")

File already exists
File already exists
No fasta found for prefix: PHAGE__GCA-031321885-1
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists


Running ART illumina to simulate reads

In [13]:
import os
import subprocess
from pathlib import Path

fasta_dir = Path("/home/shana/nelli-genomes-db/data/gvclass_genus_1_2025-08-25/filtered-mags/")
output_dir = Path("/home/shana/nelli-genomes-db/data/gvclass_genus_1_2025-08-25/simulated_reads/")
art_executable = Path("/home/shana/art_modern/opt/build_debug_install/bin/art_modern")

output_dir.mkdir(parents=True, exist_ok=True)

#ART params
read_length = 150
coverage = 20
sequencing_system = "HS25" #HiSeq 2500
pe_frag_dist_mean = 300  # Mean fragment length for PE reads
pe_frag_dist_std_dev = 50  # Standard deviation for fragment length

for fasta_file in fasta_dir.glob("*.fna"):
    base_name = fasta_file.stem #filename w/o extension

    print(f"simulation reads for: {fasta_file.name}")

    art_command = [
        str(art_executable),
        "--mode", "wgs",  # Whole-genome sequencing mode
        "--lc", "pe",  # Paired-end reads
        "--i-file", str(fasta_file),  # Input FASTA file
        "--o-fastq", str(output_dir / base_name),  # Output prefix for FASTQ files
        "--read_len", str(read_length),  # Read length
        "--i-fcov", str(coverage),  # Coverage
        "--pe_frag_dist_mean", str(pe_frag_dist_mean),  # Mean fragment length
        "--pe_frag_dist_std_dev", str(pe_frag_dist_std_dev),  # Standard deviation
        "--builtin_qual_file", "HiSeq2500_150bp",
        "--parallel", "0"  # Use all available cores (or set to a specific number)
    ]

    subprocess.run(art_command)

    print("simulated reads saved as {output_dir / base_name}")

print("COMPLETE for ALL files")

simulation reads for: PHAGE__GCA-029870215-1.fna
Previous run crashed:
 0# 0x000077E7C2E0BCA2
 1# 0x000077E7C2242520
 2# 0x000077E7C22969FC
 3# 0x000077E7C2242476
 4# 0x000077E7C22287F3
 5# 0x000077E7C2E18DC6
 6# 0x000077E7C2E0D51F
 7# 0x000077E7C2A31B7B
 8# 0x000077E7C2A35790
 9# 0x000077E7C2A38CFD
10# 0x00005AABE351A60B
11# 0x000077E7C2229D90
12# 0x000077E7C2229E40
13# 0x00005AABE3519EA5



[2025-09-03 16:12:39.391943] info: YuZJ Modified ART_Illumina (art_modern v. 1.1.4)
[2025-09-03 16:12:39.392009] info: Based on: v. 2008-2016, Q Version 2.5.8 (June 6, 2016)
[2025-09-03 16:12:39.392013] info: Originally written by: Weichun Huang <whduke@gmail.com>
[2025-09-03 16:12:39.392017] info: Modified by: YU Zhejian <yuzj25@seas.upenn.edu>
[2025-09-03 16:12:39.392020] info: Debugging functions enabled.
[2025-09-03 16:12:39.392023] warning: ART_LOG_DIR not defined; Default to 'log.d'.
[2025-09-03 16:12:39.392054] warning: MPI not found! Cross-node parallelism disabled.
[2025-09-03 16:12:39.465379] info: ARGS: /home/shana/art_modern/opt/build_debug_install/bin/art_modern --mode wgs --lc pe --i-file /home/shana/nelli-genomes-db/data/gvclass_genus_1_2025-08-25/filtered-mags/PHAGE__GCA-029870215-1.fna --o-fastq /home/shana/nelli-genomes-db/data/gvclass_genus_1_2025-08-25/simulated_reads/PHAGE__GCA-029870215-1 --read_len 150 --i-fcov 20 --pe_frag_dist_mean 300 --pe_frag_dist_std_dev 50

simulated reads saved as {output_dir / base_name}
simulation reads for: EUK__GCA-902498725-1.fna


[2025-09-03 16:12:41.803709] info: All writers added
[2025-09-03 16:12:41.804085] info: Starting simulation for job 6 with 4 contigs
[2025-09-03 16:12:41.804173] info: Starting simulation for job 4 with 4 contigs
[2025-09-03 16:12:41.804198] info: Finished simulation for job 6 with 0 reads (mean depth=0) generated.
[2025-09-03 16:12:41.804285] info: Starting simulation for job 5 with 4 contigs
[2025-09-03 16:12:41.804311] info: Starting simulation for job 2 with 4 contigs
[2025-09-03 16:12:41.804345] info: Starting simulation for job 1 with 4 contigs
[2025-09-03 16:12:41.804363] info: Starting simulation for job 7 with 4 contigs
[2025-09-03 16:12:41.804336] info: Finished simulation for job 4 with 0 reads (mean depth=0) generated.
[2025-09-03 16:12:41.804376] info: Finished simulation for job 5 with 0 reads (mean depth=0) generated.
[2025-09-03 16:12:41.804420] info: Finished simulation for job 1 with 0 reads (mean depth=0) generated.
[2025-09-03 16:12:41.804336] info: Starting simulat

simulated reads saved as {output_dir / base_name}
simulation reads for: PHAGE__GCA-027582475-1.fna


...

[2025-09-03 16:12:52.131899] info: All writers added
[2025-09-03 16:12:52.132319] info: Starting simulation for job 1 with 1 contigs
[2025-09-03 16:12:52.132511] info: Starting simulation for job 4 with 1 contigs
[2025-09-03 16:12:52.132538] info: Starting simulation for job 5 with 1 contigs
[2025-09-03 16:12:52.132526] info: Starting simulation for job 3 with 1 contigs
[2025-09-03 16:12:52.132599] info: Starting simulation for job 2 with 1 contigs
[2025-09-03 16:12:52.132679] info: Starting simulation for job 6 with 1 contigs
[2025-09-03 16:12:52.132695] info: Starting simulation for job 7 with 1 contigs
[2025-09-03 16:12:52.132782] info: Starting simulation for job 8 with 1 contigs
[2025-09-03 16:12:52.132804] info: Starting simulation for job 10 with 1 contigs
[2025-09-03 16:12:52.132905] info: Starting simulation for job 9 with 1 contigs
[2025-09-03 16:12:52.133026] info: Starting simulation for job 15 with 1 contigs
[2025-09-03 16:12:52.133050] info: Starting simulation for job 11

simulated reads saved as {output_dir / base_name}
simulation reads for: EUK__GCA-046252445-1.fna


[2025-09-03 16:12:54.197243] info: All writers added
[2025-09-03 16:12:54.198085] info: Starting simulation for job 15 with 26 contigs
[2025-09-03 16:12:54.198121] info: Starting simulation for job 1 with 26 contigs
[2025-09-03 16:12:54.198186] info: Starting simulation for job 2 with 26 contigs
[2025-09-03 16:12:54.198374] info: Starting simulation for job 5 with 26 contigs
[2025-09-03 16:12:54.198527] info: Starting simulation for job 6 with 26 contigs
[2025-09-03 16:12:54.198553] info: Starting simulation for job 8 with 26 contigs
[2025-09-03 16:12:54.198578] info: Starting simulation for job 12 with 26 contigs
[2025-09-03 16:12:54.198631] info: Starting simulation for job 9 with 26 contigs
[2025-09-03 16:12:54.198645] info: Starting simulation for job 14 with 26 contigs
[2025-09-03 16:12:54.198678] info: Starting simulation for job 4 with 26 contigs
[2025-09-03 16:12:54.198694] info: Starting simulation for job 13 with 26 contigs
[2025-09-03 16:12:54.198715] info: Starting simulati

simulated reads saved as {output_dir / base_name}
COMPLETE for ALL files


[2025-09-03 16:12:55.937260] info: Job pool stopped.
[2025-09-03 16:12:55.947662] info: FASTQ LockFreeIO: Writer to '/home/shana/nelli-genomes-db/data/gvclass_genus_1_2025-08-25/simulated_reads/EUK__GCA-046252445-1' closed.
[2025-09-03 16:12:55.947682] info: FASTQ LockFreeIO: Finished, consuming 1,792 reads and writes 1,792 reads.
[2025-09-03 16:12:55.947692] info: FASTQ LockFreeIO: N. Waitings (I/ONotFull/OEmpty): 0 / 500(2.01491%) / 24,315(97.9851%).
[2025-09-03 16:12:55.947703] info: FASTQ LockFreeIO: 1.25MB written in 1.75 seconds. Speed: 731.72KB/s.
[2025-09-03 16:12:55.947748] info: Output dispatchers cleared.
[2025-09-03 16:12:55.947752] info: Generator finished.
[2025-09-03 16:12:55.993434] info: All reads generated.
[2025-09-03 16:12:55.993458] info: Time spent: 2.058s wall, 7.940s user + 0.330s system = 8.270s CPU (401.8%)
[2025-09-03 16:12:55.993476] info: EXIT


Quality Control done using FastQC, results in simulated-data dir

Combining genomes to metagenomes done in create_metagenomes.py, results in simulated-data/metagenomes dir

Metagenomes Assembly

In [2]:
import os
import subprocess
import pandas as pd
from pathlib import Path
import glob
import shutil

In [4]:
base_dir = Path("/clusterfs/jgi/scratch/science/mgs/nelli/shana/AI-Enhanced-Software-Development/simulated-data")
metagenome_dir = base_dir / "metagenomes"
assembly_dir= base_dir / "assemblies"
assembly_dir.mkdir(exist_ok=True)

for f in metagenome_dir.glob("*.fastq"):
    print(f" {f.name}")

 metagenome_euk30_phage70_rep1.fastq
 metagenome_euk30_phage70_rep2.fastq
 metagenome_euk30_phage70_rep3.fastq
 metagenome_euk50_phage50_rep1.fastq
 metagenome_euk50_phage50_rep2.fastq
 metagenome_euk50_phage50_rep3.fastq
 metagenome_euk70_phage30_rep1.fastq
 metagenome_euk70_phage30_rep2.fastq
 metagenome_euk70_phage30_rep3.fastq


In [12]:
#assembly function
def assemble_metagenome(input_fastq, output_dir, sample_name):

    sample_output_dir = output_dir / sample_name

    cmd = ['conda', 'run', '-n', 'megahit-env', 'megahit',
    '-r', str(input_fastq), '-o', str(sample_output_dir), '--min-contig-len', '1000', 
    '--k-min', '21', '--k-max', '141', '--k-step', '20', '--memory', '0.8', '--num-cpu-threads', '4', '--verbose']

    print(f"Assembling {sample_name}...")

    try:
        result = subprocess.run(cmd, capture_output=True, text=True, check=True)
        print(f"Assembly completed successfully for {sample_name}")
        print("STDOUT:", result.stdout[-500:])  # Last 500 characters
        return True, str(sample_output_dir)
    except subprocess.CalledProcessError as e:
        print(f"Assembly failed for {sample_name}")
        print("STDERR:", e.stderr)
        return False, str(sample_output_dir)

In [6]:
# Debug: Check what's in the assemblies directory
print("Current contents of assemblies directory:")
for item in assembly_dir.iterdir():
    print(f"  {item.name} (directory: {item.is_dir()})")

print(f"\nAssembly directory exists: {assembly_dir.exists()}")
print(f"Assembly directory path: {assembly_dir}")

# Test if we can create and remove a test directory
test_dir = assembly_dir / "test_debug"
print(f"\nTesting directory creation/removal:")
print(f"Test directory exists before: {test_dir.exists()}")

test_dir.mkdir(exist_ok=True)
print(f"Test directory exists after creation: {test_dir.exists()}")

import shutil
shutil.rmtree(test_dir)
print(f"Test directory exists after removal: {test_dir.exists()}")

Current contents of assemblies directory:

Assembly directory exists: True
Assembly directory path: /clusterfs/jgi/scratch/science/mgs/nelli/shana/AI-Enhanced-Software-Development/simulated-data/assemblies

Testing directory creation/removal:
Test directory exists before: False
Test directory exists after creation: True
Test directory exists after removal: False


In [14]:
#run assemblies for all metagenomes
metagenome_files = list(metagenome_dir.glob("*.fastq"))
print(f"Found {len(metagenome_files)} metagenomes to assemble")

assembly_results = []
for metagenome_file in metagenome_files:
    sample_name = metagenome_file.stem
    print(f"\n{'='*60}")
    print(f"Assembling {sample_name}...")
    print(f"File: {metagenome_file}")

    success = assemble_metagenome(metagenome_file, assembly_dir, sample_name)
    assembly_results.append(
        {'sample': sample_name, 'input_file': str(metagenome_file), 
        'output_dir': str(assembly_dir / sample_name), 'success': success}
    )

print(f"\n{'='*60}")
print("ASSEMBLY SUMMARY")
print(f"{'='*60}")
for result in assembly_results:
    status = "✓ SUCCESS" if result['success'] else "✗ FAILED"
    print(f"{result['sample']}: {status}")

Found 9 metagenomes to assemble

Assembling metagenome_euk30_phage70_rep1...
File: /clusterfs/jgi/scratch/science/mgs/nelli/shana/AI-Enhanced-Software-Development/simulated-data/metagenomes/metagenome_euk30_phage70_rep1.fastq
Assembling metagenome_euk30_phage70_rep1...


Assembly completed successfully for metagenome_euk30_phage70_rep1
STDOUT: 

Assembling metagenome_euk30_phage70_rep2...
File: /clusterfs/jgi/scratch/science/mgs/nelli/shana/AI-Enhanced-Software-Development/simulated-data/metagenomes/metagenome_euk30_phage70_rep2.fastq
Assembling metagenome_euk30_phage70_rep2...
Assembly completed successfully for metagenome_euk30_phage70_rep2
STDOUT: 

Assembling metagenome_euk30_phage70_rep3...
File: /clusterfs/jgi/scratch/science/mgs/nelli/shana/AI-Enhanced-Software-Development/simulated-data/metagenomes/metagenome_euk30_phage70_rep3.fastq
Assembling metagenome_euk30_phage70_rep3...
Assembly completed successfully for metagenome_euk30_phage70_rep3
STDOUT: 

Assembling metagenome_euk50_phage50_rep1...
File: /clusterfs/jgi/scratch/science/mgs/nelli/shana/AI-Enhanced-Software-Development/simulated-data/metagenomes/metagenome_euk50_phage50_rep1.fastq
Assembling metagenome_euk50_phage50_rep1...
Assembly completed successfully for metagenome_euk50_phage50_

In [ ]:
#assembly quality assessment
def get_assembly_stats(assembly_file):
    if not assembly_file.exists():
        return None
    stats = {
        'file': assembly_file.name, 'total_contigs': 0, 'total_length': 0, 'n50': 0, 
        'n90': 0, 'max_length': 0, 'min_length': float('inf')
    }

    contig_lengths = []

    with open(assembly_file, 'r') as f:
        current_length = 0
        for line in f:
            if line.startswith('>'):
                if current_length > 0:
                    contig_lengths.append(current_length)
                    stats['total_length'] += current_length
                    stats['total_contigs'] += 1
                    stats['max_length'] = max(stats['max_length'], current_length)
                    stats['min_length'] = min(stats['min_length'], current_length)
                current_length = 0
            else:
                current_length += len(line.strip())
        
        #last contig
        if current_length > 0:
            contig_lengths.append(current_length)
            stats['total_length'] += current_length
            stats['total_contigs'] += 1
            stats['max_length'] = max(stats['max_length'], current_length)
            stats['min_length'] = min(stats['min_length'], current_length)

    if contig_lengths:
        #calculate N50 and N90
        contig_lengths.sort(reverse=True)
        cumulative_length = 0
        half_length = stats['total_length'] / 2
        ninety_percent_length = stats['total_length'] * 0.9

        for length in contig_lengths:
            cumulative_length += length
            if cumulative_length >= half_length and stats['n50'] == 0:
                stats['n50'] = length
            if cumulative_length >= ninety_percent_length and stats['n90'] == 0:
                stats['n90'] = length
                break
        
        if stats['min_length'] == float('inf'):
            stats['min_length'] = 0
    
    return stats

assembly_stats = []

for result in assembly_results:
    if result['success']:
        assembly_file = Path(result['output_dir']) / 'final.contigs.fa'
        stats = get_assembly_stats(assembly_file)
        if stats:
            stats['sample'] = result['sample']
            assembly_stats.append(stats)

#create df and display
if assembly_stats:
    df_stats = pd.DataFrame(assembly_stats)
    df_stats = df_stats[['sample', 'total_contigs', 'total_length', 'n50', 'n90', 'max_length', 'min_length']]

    print("ASSEMBLY STATISTICS")
    print("=" * 80)
    print(df_stats.to_string(index=False, formatters={
        'total_length': '{:,}'.format,
        'n50': '{:,}'.format,
        'n90': '{:,}'.format,
        'max_length': '{:,}'.format,
        'min_length': '{:,}'.format
    }))

    stats_file = assembly_dir / "assembly_statistics.csv"
    df_stats.to_csv(stats_file, index=False)
    print(f"\nAssembly statistics saved to {stats_file}")
else:
    print("No assemblies were successful")


ASSEMBLY STATISTICS
                       sample  total_contigs total_length   n50   n90 max_length min_length
metagenome_euk30_phage70_rep1             19       58,652 5,684 1,160     11,817      1,012
metagenome_euk30_phage70_rep2             19       58,652 5,684 1,160     11,817      1,012
metagenome_euk30_phage70_rep3             19       58,652 5,684 1,160     11,817      1,012
metagenome_euk50_phage50_rep1             19       58,652 5,684 1,160     11,817      1,012
metagenome_euk50_phage50_rep2             19       58,652 5,684 1,160     11,817      1,012
metagenome_euk50_phage50_rep3             19       58,652 5,684 1,160     11,817      1,012
metagenome_euk70_phage30_rep1             19       58,652 5,684 1,160     11,817      1,012
metagenome_euk70_phage30_rep2             19       58,652 5,684 1,160     11,817      1,012
metagenome_euk70_phage30_rep3             19       58,652 5,684 1,160     11,817      1,012

Assembly statistics saved to /clusterfs/jgi/scratch/science

In [ ]:
#skipping cell 6, 7, and 8

Getting taxonomic classifications using Kraken (done in terminal)